In [1]:
using Revise
using KLIEPInference
using Distributions
using LinearAlgebra
using BenchmarkTools

In [141]:
p = 100
θx = chain(p, 10, 0.2, 0.4, 0)
θy = copy(θx)
removeEdges!(θy, 4)
Δ = θx - θy
ind_nz = findall(!iszero, Δ)

4-element Array{Int64,1}:
    6
 1485
 1540
 4371

In [142]:
Δ[ind_nz]

4-element Array{Float64,1}:
 0.33825496914877096
 0.3672520676397463 
 0.22772231427944836
 0.2007802021942048 

In [112]:
unpack(θx)

50×50 Array{Float64,2}:
 0.0       0.516424  0.0       0.0       …  0.0       0.0       0.0     
 0.516424  0.0       0.408377  0.0          0.0       0.0       0.0     
 0.0       0.408377  0.0       0.476824     0.0       0.0       0.0     
 0.0       0.0       0.476824  0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.458486     0.0       0.0       0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 ⋮                         

In [85]:
unpack(θy)

5×5 Array{Float64,2}:
 0.0       0.497204  0.0       0.0       0.0
 0.497204  0.0       0.0       0.0       0.0
 0.0       0.0       0.0       0.595112  0.0
 0.0       0.0       0.595112  0.0       0.0
 0.0       0.0       0.0       0.0       0.0

In [143]:
nx = 500
ny = 500
spl = IsingSampler(θx; thin=1000)
X = rand(spl, nx)
spl = IsingSampler(θy; thin=1000)
Y = rand(spl, ny)
Ψx = Ψising(X)
Ψy = Ψising(Y);

In [144]:
# @show σ = maximum( sqrt.( nx * var(Ψx; dims=2) + ny * var(Ψy; dims=2) ) ) / sqrt(nx + ny)
m = p * (p - 1) / 2.
# @show λ = 2. * sqrt(log(m) / nx)
@show λ = 1.2 * sqrt(2. * log(m) / nx)
θhat = spKLIEP(Ψx, Ψy, λ, CD_KLIEP())

λ = 1.2 * sqrt((2.0 * log(m)) / nx) = 0.22136201762732774


4950-element ProximalBase.SparseIterate{Float64,1} with 6 stored entries:
  [6   ]  =  0.0639473
  [691 ]  =  0.0255212
  [1451]  =  -0.00914313
  [1485]  =  0.124683
  [4371]  =  0.0837854
  [4418]  =  0.0221621


In [123]:
H = KLIEP_Hessian(θhat, Ψy);

In [127]:
@time Hinv_row(H, 6, 1.2 * sqrt(2. * log(p) / ny))

  0.518345 seconds (41 allocations: 504.391 KiB)


4950-element ProximalBase.SparseIterate{Float64,1} with 5 stored entries:
  [5   ]  =  -0.17333
  [6   ]  =  1.02486
  [9   ]  =  -0.118991
  [2497]  =  0.00778956
  [3225]  =  -0.000652763
